# Comparing Different Models

In [ ]:
import numpy as np
import torch
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

## Loading Data

First we need to load the testing datasets for each model

In [ ]:
# load datasets from the folder
data_dir = 'data/'
image_sizes = ['4-4', '8-8', '16-16', '25-25', '32-32']
dirs = [data_dir + size + '/temp_class/test' for size in image_sizes]

# define the transforms
dset_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()])

# load the datasets
dsets = {y: datasets.ImageFolder(x, dset_transform) for x in dirs for y in image_sizes}


## Load the models

In [ ]:
model_dir = 'models/'
model_names = [size + '-NN.pth' for size in image_sizes]
model_paths = [model_dir + name for name in model_names]

# load the models
models = {y: torch.load(x) for x in model_paths for y in image_sizes}